# FINN -  Functional Verification |  UNSW_NB15
------------------------------------------------------------------------------------------------------------------

**Important: This notebook depends on the cyberbersecurity_Brevitas_1bit notebook, because we are using models that were created by this notebook. So please make sure the needed .onnx files are generated to run this notebook.**

In this notebook we will show how to verify the model created by Brevitas. If this model is sucessfully verified, it can be taken all the way down to be implemented on the board.

In [1]:
!pip install --user pandas
!pip install --user scikit-learn

import onnx 
import torch 

## Outline
-------------
1. [Brevitas import and visualization](#brevitas_import_visualization)
2. [Network preperations: Tidy up transformations](#network_preparations)
3. [Simulate the model node by node and get all outputs](#simulate_node) 
4. [Compare with Brevitas](#compare_brevitas)

# 1. Brevitas import and visualization <a id="brevitas_import_visualization"></a>

Now that we have the model in .onnx format, we can work with it using FINN. For that FINN ModelWrapper is used. It is a wrapper around the ONNX model which provides several helper functions to make it easier to work with the model.

In [2]:
from finn.core.modelwrapper import ModelWrapper

model_file_path = "brevitas_w1_a1NSW_NB15_model.onnx"
model_for_sim = ModelWrapper(model_file_path)

To visualize the exported model, Netron can be used. Netron is a visualizer for neural networks and allows interactive investigation of network properties. For example, you can click on the individual nodes and view the properties.

In [3]:
from finn.util.visualization import showSrc, showInNetron
showInNetron(model_file_path)

Serving 'brevitas_w1_a1NSW_NB15_model.onnx' at http://0.0.0.0:8081


# 2. Network preperations: Tidy up transformations <a id="network_preparations"></a>

This section deals with some basic transformations, which are applied to the model like a kind of "tidy-up" to make it easier to be processed.

In the first two transformations (`GiveUniqueNodeNames`, `GiveReadableTensorNames`) the nodes in the graph are first given unique (by enumeration) names, then the tensors are given human-readable names (based on the node names). The following two transformations (`InferShapes`, `InferDataTypes`) derive the shapes and data types of the tensors from the model properties and set them in the `ValueInfo` of the model. These transformations can almost always be applied without negative effects and do not affect the structure of the graph, ensuring that all the information needed is available.

The next listed transformation is `FoldConstants`, which performs constant folding. It identifies a node with constant inputs and determines its output. The result is then set as constant-only inputs for the following node and the old node is removed. Although this transformation changes the structure of the model, it is a transformation that is usually always desired and can be applied to any model. And finally, we have `RemoveStaticGraphInputs` to remove any top-level graph inputs that already have ONNX initializers associated with them.

In [4]:
from finn.transformation.general import GiveReadableTensorNames, GiveUniqueNodeNames, RemoveStaticGraphInputs
from finn.transformation.infer_shapes import InferShapes
from finn.transformation.infer_datatypes import InferDataTypes
from finn.transformation.fold_constants import FoldConstants

model_for_sim = model_for_sim.transform(InferShapes())
model_for_sim = model_for_sim.transform(FoldConstants())
model_for_sim = model_for_sim.transform(GiveUniqueNodeNames())
model_for_sim = model_for_sim.transform(GiveReadableTensorNames())
model_for_sim = model_for_sim.transform(InferDataTypes())
model_for_sim = model_for_sim.transform(RemoveStaticGraphInputs())

# 3. Load the UNSW_NB15 Test set <a id="Load_test_set"></a>

The UNSW_NB15_quantized class loads the csv file with the dataset, quantizes it and then returns it. Its values range between {0,1}.

In [5]:
from torch.utils.data import DataLoader, Dataset
from dataloader_quantized import UNSW_NB15_quantized

test_quantized_dataset = UNSW_NB15_quantized(file_path_train='UNSW_NB15_training-set.csv', \
                                              file_path_test = "UNSW_NB15_testing-set.csv", \
                                              train=False)
input_tensor = test_quantized_dataset.data[:,:-1]


torch.Size([82332, 594])


torch.Size([82332, 593])

# 4. Simulate the model node by node and get all outputs <a id="simulate_node"></a>

The UNSW_NB15_quantized class allows to create an input tensor which will be passed to the execution function.

This execution function  `execute_onnx` from `onnx_exec` library is applied to the model. The model is then simulated node by node and the result is stored in a context dictionary, which contains the values of each tensor at the end of the execution. To get the result, only the output tensor has to be extracted.

The output is extracted for each one of the input, one by one, and saved inside an array which we will compare against brevitas output to see if both models (created iwth FINN and with Brevitas) are outputting the same.

This next step might take several hours to be processed.

In [ ]:
import finn.core.onnx_exec as oxe
import numpy as np

prev_i = 0
for i in range(1, input_tensor.shape[0], 1): 
    np_array_k_rows = input_tensor[prev_i:i]
    # transform the input from {0,+1} to {-1,+1}
    np_array_k_rows = np_array_k_rows * 2.0 - torch.tensor([1.0])
    np_array_k_rows = np_array_k_rows.detach().numpy() # select only the first k rows
    
    input_dict = {"global_in": np_array_k_rows} # create the dictionary
    
    output_dict = oxe.execute_onnx(model_for_sim, input_dict) #execute each node
    output_pysim = output_dict[list(output_dict.keys())[0]] #get the output
    
    finn_partial_array_after_sigmoid = torch.sigmoid(torch.from_numpy(output_pysim)).detach().numpy()
    finn_partial_array_after_sigmoid = ((finn_partial_array_after_sigmoid > 0.5) * 1 ) *2 - 1 #shift to {-1,+1}
    
    if prev_i == 0:
        finn_array_after_sigmoid  = finn_partial_array_after_sigmoid
        
    else:
        finn_array_after_sigmoid = np.append(finn_array_after_sigmoid, finn_partial_array_after_sigmoid, axis=0)

    prev_i = i
    if i%10_000 == 0:
        print(i)

10000


# 5. Compare with Brevitas <a id="compare_brevitas"></a>

Let's see if the output of the FINN model matches the output of the Brevitas model. 

In [ ]:
from numpy import genfromtxt

brevitas_array_after_sigmoid = genfromtxt('brevitas_1_bit_model_output.csv', delimiter=',').reshape(-1,1)
number_differences = (finn_array_after_sigmoid != brevitas_array_after_sigmoid).sum()
if number_differences == 0:
    print("Congrats! Both models are outputting the same thing. This verifies their functionality!!")
else:
    print("Something went wrong... there are", number_differences, "differences out of the", input_tensor.shape[0], "total cases.")

If the amount of differences is zero then both models are outputting the same thing. This verifies the functionality! 

Now that we verified that both models output the same, we need to set the tensor datatype of the model and then save it.

In [ ]:
from finn.core.datatype import DataType
global_inp_name = model_for_sim.graph.input[0].name
model_for_sim.set_tensor_datatype(global_inp_name,  DataType.BIPOLAR)
model_for_sim.save(model_file_path)

Great! 
Now that you have verified the FINN model, you can take your model all the way down to implementing it on the board.